# ct.gov BiViewer Demo

This notebook attempts to make viewing a ct.gov entry and its full-text PDF easy for the purposes of data exploration. Scroll down to the section *Pop Open XML Files for Both Pubmed and `ct.gov` in a Web Browser* for the functionality.

That section's code operates as follows:

- Get an instance of `biviewer.ClinicalTrialsBiViewer`
- Get a study
- Extract the pmid and nct IDs
- Pop open a web browser with the following two tabs:
    - The `ct.gov` entry
    - The pubmed page for the full-text pdf

### Play Around with Cochrane

In [5]:
from cochranenlp.readers import biviewer

p = biviewer.PDFBiViewer()

In [16]:
def cochranes(p):
    for i in range(100000):
        yield p[i].cochrane['CHARACTERISTICS']
        
cochrane = cochranes(p)

In [59]:
next(cochrane)

{'CHAR_INTERVENTIONS': 'Participants were randomly assigned to two groups. \n Group 1: IPC (n = 50), 10 minutes of clamping. \n Group 2: No IPC (n = 51).',
 'CHAR_METHODS': 'Randomised clinical trial \n Generation of the allocation sequence: computer generated (adequate). Allocation concealment: sealed envelope technique (adequate). Blinding: inadequate. Incomplete outcome data addressed: yes. Free of selective reporting: yes. Free from baseline imbalance: yes. Free from early stopping bias: yes. Free from bias due to blocked randomisation: yes.',
 'CHAR_NOTES': '',
 'CHAR_OUTCOMES': 'The outcomes reported were mortality, morbidity, IPF, PGNF, re-transplantation, ITU stay, hospital stay, and liver function tests.',
 'CHAR_PARTICIPANTS': 'Country: USA. Number randomised: 101. Donor mean age: 46 years. Recipient mean age: 53 years. Donor females: 41 (40.6%). Recipient females: 24 (23.8%). Living or cadaveric donors: cadaveric. \n Inclusion criteria \n \n Cadaveric donors. \n Adult recipi

### Pubmed Viewer

In [79]:
from collections import namedtuple
from cochranenlp.readers import biviewer

Pubmed_Info = namedtuple('Pubmed_Info', ['abstract', 'language', 'linkedIds', 'mesh', 'pmid', 'title', 'filename'])
c = biviewer.ClinicalTrialsBiViewer()

def pubmed_sections(study):
    """Yields all pubmed sections for a particular study
    
    Study is a BiViewer_View(clinicaltrials, pubmed) tuple
    
    """
    for section in dir(study.pubmed):
        yield study.pubmed.text_filtered_all(section)
        
    yield study.pubmed.filename

def pubmeds_generator(c, field):
    """Yields pubmed info for all studies
    
    c is a biviewer.ClinicalTrialsBiViewer
    
    This generator is not meant to be exhausted. Rather it is meant to be called one-by-one with next()
    
    """
    for i in range(1000):
        study = c[i]
        
        pi = Pubmed_Info(*pubmed_sections(study))
        
        yield pi._asdict()[field]

### `ct.gov` Viewer

In [105]:
from collections import namedtuple
from cochranenlp.readers import biviewer

CT_Info = namedtuple('CT_Info',
                         ['arms',
                          'design',
                          'interventions',
                          'nct_id',
                          'num_arms',
                          'outcomes',
                          'pmids',
                          'population',
                          'resultspmids',
                          'title',
                          'filename'])

c = biviewer.ClinicalTrialsBiViewer()

def ct_sections(study):
    """Yields all ct.gov sections for a particular study
    
    Study is a BiViewer_View(clinicaltrials, pubmed) tuple
    
    """
    for section in dir(study.clinicaltrials):
        yield study.clinicaltrials.text_filtered_all(section)
        
    yield study.pubmed.filename

def cts_generator(c, field):
    """Yields ct.gov info for all studies
    
    c is a biviewer.ClinicalTrialsBiViewer
    
    This generator is not meant to be exhausted. Rather it is meant to be called one-by-one with next()
    
    """
    for i in range(1000):
        study = c[i]
        
        pi = CT_Info(*ct_sections(study))
        
        yield pi._asdict()[field]

### Pop Open XML Files for Both Pubmed and `ct.gov` in a Web Browser

In [86]:
import random

from collections import namedtuple
from cochranenlp.readers import biviewer

Pubmed_CT = namedtuple('CT_Info', ['pmid', 'outcomes'])

c = biviewer.ClinicalTrialsBiViewer()


def biviewer_generator(c, start=0, end=1000, rand=False):
    """Yields pops open Google Chrome on both XML pages for a pubmed and ct.gov study
    
    c is a biviewer.ClinicalTrialsBiViewer
    
    This generator is not meant to be exhausted. Rather it is meant to be called one-by-one with next()
    
    """
    rand = random.randint(start, end) if not rand else rand
    
    for i in range(start, end):
        study = c[i]
        
        pubmed, ct = study.pubmed, study.clinicaltrials
        pmid = pubmed.text_filtered_all('pmid')[0]
        nct_id = ct.text_filtered_all('nct_id')[0]
        
        pm_url = 'https://www.google.com/search?q=pmid+' + pmid + '&btnI=I' # I'm Feeling Lucky
        ct_url = 'https://clinicaltrials.gov/show/' + nct_id
        
        !google-chrome "{ct_url}"
        !google-chrome "{pm_url}"
        
        yield ct.text_filtered_all('outcomes')

bg = biviewer_generator(c)

In [89]:
outcomes = next(bg)

outcomes[0].split('\n')

Created new window in existing browser session.
Created new window in existing browser session.


['Primary ',
 '         Number of Participants With an Immune Response to Tumor-specific and Non-tumor Specific Peptides During a Period of Immune Reconstitution ',
 "         Immune response was defined as a percent specific lysis of >10% following challenge with peptide pulsed targets, or interferon gamma production following challenge with peptide pulsed targets >2-fold that found with no-peptide controls or a proliferation index >3.0. Tumor specific peptides: Ewings sarcoma Type 1: EF-1 (EWS/FLI-1)*SSSYGQQN/PSYDSVRRGA,Ewing's Sarcoma Type 2: EF-2 (EWS/FLI-2)* SSSYGQ/QSSLLAYNT, Alveolar rhabdomyosarcoma: PXFK (PAX3/FKHR)\xe2\x80\xa0 TIGNGLSPQ/NSIRHNLSL. Non-tumor specific peptide:HPV16E7 MLDLQPETT-MET-9-THR. See protocol link module for additional information re: peptides. ",
 '         20 weeks post vaccination ',
 '         No ',
 '         ',
 '           ',
 '             Peptide Vaccine/Autologous T Cell Transplant/Indinavir Therapy ',
 '             Patients receive oral indin